#### ML Paper Social Media Post Generator

This notebook creates a multi-agent system that:
1. Takes a Machine Learning paper as input
2. Generates a social media post about the paper
3. Verifies the post for correctness and platform-specific style

## System Architecture

- **Content Team**: Analyzes paper and creates initial post
- **Verification Team**: Checks accuracy and platform compliance
- **Meta-Supervisor**: Coordinates between teams


## Dependencies and Setup


In [1]:
import os
import getpass
import functools
import operator
from typing import Annotated, List, TypedDict, Optional, Dict

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# LangGraph imports
from langgraph.graph import START, END, StateGraph

# Community tools
from langchain_community.tools import ArxivQueryRun


In [2]:
# Set up API keys
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")


## Helper Functions


In [3]:
def agent_node(state, agent, name):
    """Helper function to create agent nodes"""
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

def create_agent(
    llm: ChatOpenAI,
    tools: list,
    system_prompt: str,
) -> AgentExecutor:
    """Create a function-calling agent"""
    system_prompt += ("\nWork autonomously according to your specialty, using the tools available to you."
    " Do not ask for clarification."
    " Your other team members will collaborate with you with their own specialties."
    " You are chosen for a reason!")
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    agent = create_openai_functions_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

def create_team_supervisor(llm: ChatOpenAI, system_prompt: str, members: List[str]):
    """Create a supervisor agent for routing"""
    options = ["FINISH"] + members
    function_def = {
        "name": "route",
        "description": "Select the next role.",
        "parameters": {
            "title": "routeSchema",
            "type": "object",
            "properties": {
                "next": {
                    "title": "Next",
                    "anyOf": [{"enum": options}],
                },
            },
            "required": ["next"],
        },
    }
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        ("system", "Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}"),
    ]).partial(options=str(options), team_members=", ".join(members))
    
    return (
        prompt
        | llm.bind_functions(functions=[function_def], function_call="route")
        | JsonOutputFunctionsParser()
    )


## Tools Creation


In [4]:
# Arxiv tool for fetching papers
arxiv_tool = ArxivQueryRun()

@tool
def analyze_paper_content(
    paper_title: Annotated[str, "Title of the ML paper to analyze"],
    paper_summary: Annotated[str, "Summary or abstract of the paper"]
) -> str:
    """Analyze ML paper content and extract key insights for social media"""
    analysis = f"""
    Paper Analysis:
    Title: {paper_title}
    
    Key Points Extracted:
    - Main contribution: {paper_summary[:200]}...
    - Technical innovation level: High
    - Practical applications: Identified
    - Target audience: ML practitioners and researchers
    
    Recommendation: This paper is suitable for social media sharing due to its relevance and impact.
    """
    return analysis

@tool
def create_social_post(
    platform: Annotated[str, "Social media platform (twitter, linkedin, instagram)"],
    content_summary: Annotated[str, "Summary of the paper content"],
    key_points: Annotated[str, "Key technical points to highlight"]
) -> str:
    """Create a social media post based on paper analysis"""
    if platform.lower() == "twitter":
        post = f"🧠 New ML Paper Alert! \n\n{content_summary[:150]}\n\n{key_points[:100]}\n\n#MachineLearning #AI #Research"
    elif platform.lower() == "linkedin":
        post = f"📊 Exciting developments in Machine Learning!\n\n{content_summary}\n\nKey insights:\n{key_points}\n\nWhat are your thoughts on this research direction?\n\n#MachineLearning #ArtificialIntelligence #Research #Innovation"
    else:
        post = f"🚀 ML Research Highlight\n\n{content_summary}\n\n💡 {key_points}\n\n#ML #AI #TechResearch"
    
    return post

@tool
def verify_technical_accuracy(
    original_paper_info: Annotated[str, "Original paper information"],
    social_post: Annotated[str, "Generated social media post"]
) -> str:
    """Verify that the social post accurately represents the paper"""
    verification = f"""
    Technical Accuracy Check:
    ✅ Claims in post match paper content
    ✅ No technical misrepresentations found
    ✅ Appropriate level of simplification for general audience
    ✅ Key contributions properly highlighted
    
    Verification Status: APPROVED
    Confidence: High
    """
    return verification

@tool
def verify_platform_compliance(
    platform: Annotated[str, "Target social media platform"],
    post_content: Annotated[str, "Social media post content"]
) -> str:
    """Verify that the post fits platform style and requirements"""
    post_length = len(post_content)
    
    if platform.lower() == "twitter":
        status = "APPROVED" if post_length <= 280 else "NEEDS REVISION - Too long"
        guidelines = "Character limit: 280, Uses hashtags, Engaging tone"
    elif platform.lower() == "linkedin":
        status = "APPROVED" if post_length <= 3000 else "NEEDS REVISION - Too long"
        guidelines = "Professional tone, Engagement question, Industry hashtags"
    else:
        status = "APPROVED" if post_length <= 2200 else "NEEDS REVISION - Too long"
        guidelines = "Visual elements mentioned, Trendy hashtags, Concise format"
    
    verification = f"""
    Platform Compliance Check for {platform.upper()}:
    
    Post Length: {post_length} characters
    Platform Guidelines: {guidelines}
    Status: {status}
    
    Style Assessment: Appropriate tone and format for {platform}
    """
    return verification


## State Definitions


In [5]:
# Content Team State
class ContentTeamState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    paper_info: str
    platform: str
    next: str

# Verification Team State  
class VerificationTeamState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    paper_info: str
    platform: str
    social_post: str
    next: str

# Main System State
class MainState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    paper_query: str
    platform: str
    final_post: str
    next: str


## Content Creation Team


In [6]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini")

# Content Team Agents
paper_researcher = create_agent(
    llm,
    [arxiv_tool, analyze_paper_content],
    "You are a research assistant specializing in finding and analyzing ML papers. Your job is to fetch paper information and extract key insights for social media content."
)

content_creator = create_agent(
    llm,
    [create_social_post],
    "You are a social media content creator specializing in technical content. Create engaging posts that make ML research accessible to broader audiences while maintaining accuracy."
)

# Content Team Nodes
researcher_node = functools.partial(agent_node, agent=paper_researcher, name="Researcher")
creator_node = functools.partial(agent_node, agent=content_creator, name="ContentCreator")

# Content Team Supervisor
content_supervisor = create_team_supervisor(
    llm,
    "You are supervising a content creation team. The Researcher finds and analyzes papers, and the ContentCreator makes social posts. Route tasks appropriately and finish when a post is created.",
    ["Researcher", "ContentCreator"]
)


/var/folders/kz/6q4vt7hx2c70b45zm9kg3kjr0000gn/T/ipykernel_86213/606616605.py:54: LangChainDeprecationWarning: The method `BaseChatOpenAI.bind_functions` was deprecated in langchain-openai 0.2.1 and will be removed in 1.0.0. Use :meth:`~langchain_openai.chat_models.base.ChatOpenAI.bind_tools` instead.
  | llm.bind_functions(functions=[function_def], function_call="route")


In [7]:
# Build Content Team Graph
content_graph = StateGraph(ContentTeamState)

content_graph.add_node("Researcher", researcher_node)
content_graph.add_node("ContentCreator", creator_node)
content_graph.add_node("supervisor", content_supervisor)

# Add edges
content_graph.add_edge("Researcher", "supervisor")
content_graph.add_edge("ContentCreator", "supervisor")
content_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {"Researcher": "Researcher", "ContentCreator": "ContentCreator", "FINISH": END}
)

content_graph.set_entry_point("supervisor")
compiled_content_graph = content_graph.compile()


## Verification Team


In [8]:
# Verification Team Agents
technical_verifier = create_agent(
    llm,
    [verify_technical_accuracy],
    "You are a technical accuracy verifier. Your job is to ensure social media posts accurately represent the original research without misrepresenting technical details."
)

platform_verifier = create_agent(
    llm,
    [verify_platform_compliance],
    "You are a social media platform compliance expert. Check that posts meet platform requirements, style guidelines, and character limits."
)

# Verification Team Nodes
technical_node = functools.partial(agent_node, agent=technical_verifier, name="TechnicalVerifier")
platform_node = functools.partial(agent_node, agent=platform_verifier, name="PlatformVerifier")

# Verification Team Supervisor
verification_supervisor = create_team_supervisor(
    llm,
    "You are supervising a verification team. TechnicalVerifier checks accuracy, PlatformVerifier checks platform compliance. Both checks must pass before finishing.",
    ["TechnicalVerifier", "PlatformVerifier"]
)


In [9]:
# Build Verification Team Graph
verification_graph = StateGraph(VerificationTeamState)

verification_graph.add_node("TechnicalVerifier", technical_node)
verification_graph.add_node("PlatformVerifier", platform_node)
verification_graph.add_node("supervisor", verification_supervisor)

# Add edges
verification_graph.add_edge("TechnicalVerifier", "supervisor")
verification_graph.add_edge("PlatformVerifier", "supervisor")
verification_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {"TechnicalVerifier": "TechnicalVerifier", "PlatformVerifier": "PlatformVerifier", "FINISH": END}
)

verification_graph.set_entry_point("supervisor")
compiled_verification_graph = verification_graph.compile()


## Main System Integration


In [10]:
# Helper functions for main system
def get_last_message(state: MainState) -> str:
    return state["messages"][-1].content

def join_graph(response: dict):
    return {"messages": [response["messages"][-1]]}

def create_content_input(state: MainState) -> ContentTeamState:
    return {
        "messages": [HumanMessage(content=f"Create a social media post about: {state['paper_query']} for platform: {state['platform']}")],
        "paper_info": state["paper_query"],
        "platform": state["platform"],
        "next": ""
    }

def create_verification_input(state: MainState) -> VerificationTeamState:
    # Extract the social post from the last message
    last_message = state["messages"][-1].content
    return {
        "messages": [HumanMessage(content=f"Verify this social post: {last_message}")],
        "paper_info": state["paper_query"],
        "platform": state["platform"],
        "social_post": last_message,
        "next": ""
    }

# Meta-supervisor
meta_supervisor = create_team_supervisor(
    llm,
    "You coordinate between ContentTeam and VerificationTeam. ContentTeam creates posts, VerificationTeam checks them. Always verify after content creation.",
    ["ContentTeam", "VerificationTeam"]
)


In [11]:
# Build Main System Graph
main_graph = StateGraph(MainState)

main_graph.add_node("ContentTeam", create_content_input | compiled_content_graph | join_graph)
main_graph.add_node("VerificationTeam", create_verification_input | compiled_verification_graph | join_graph)
main_graph.add_node("supervisor", meta_supervisor)

# Add edges
main_graph.add_edge("ContentTeam", "supervisor")
main_graph.add_edge("VerificationTeam", "supervisor")
main_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {"ContentTeam": "ContentTeam", "VerificationTeam": "VerificationTeam", "FINISH": END}
)

main_graph.set_entry_point("supervisor")
compiled_main_graph = main_graph.compile()


## Testing the System


In [12]:
# Test the complete system
def generate_social_post(paper_query: str, platform: str = "twitter"):
    """Generate a verified social media post about an ML paper"""
    
    initial_state = {
        "messages": [HumanMessage(content=f"Generate a social media post about: {paper_query}")],
        "paper_query": paper_query,
        "platform": platform,
        "final_post": "",
        "next": ""
    }
    
    print(f"🚀 Generating {platform} post about: {paper_query}")
    print("=" * 60)
    
    for step in compiled_main_graph.stream(initial_state, {"recursion_limit": 20}):
        if "__end__" not in step:
            for key, value in step.items():
                print(f"📍 {key}: {value}")
            print("-" * 40)
    
    return "Post generation complete!"


In [13]:
# Example 1: Twitter post about Transformers
generate_social_post("Attention is All You Need transformer architecture", "twitter")


🚀 Generating twitter post about: Attention is All You Need transformer architecture
📍 supervisor: {'next': 'ContentTeam'}
----------------------------------------
📍 ContentTeam: {'messages': [HumanMessage(content='🧠 New ML Paper Alert! \n\nTransformers revolutionize NLP with their attention mechanism. Recent research shows removing some attention layers can speed up inference in large language models with minimal performance loss! \n\n✨ Key findings:\n- Reduced latency during inference \n- Up to 1.8% performance drop for 13B Llama2 model \n- Enhances efficiency & optimizes large models!\n\n#MachineLearning #AI #Transformers #LLMs #Research #Innovation', additional_kwargs={}, response_metadata={}, name='ContentCreator')]}
----------------------------------------
📍 supervisor: {'next': 'VerificationTeam'}
----------------------------------------
📍 VerificationTeam: {'messages': [HumanMessage(content="Here’s a revised version of the post to fit within Twitter's character limit:\n\n🧠 New M

'Post generation complete!'

In [14]:
# Example 2: LinkedIn post about diffusion models
generate_social_post("Denoising Diffusion Probabilistic Models", "linkedin")


🚀 Generating linkedin post about: Denoising Diffusion Probabilistic Models
📍 supervisor: {'next': 'ContentTeam'}
----------------------------------------
📍 ContentTeam: {'messages': [HumanMessage(content='📊 **Exciting developments in Machine Learning!**\n\nRician Denoising Diffusion Probabilistic Models for Sodium Breast MRI Enhancement improves image quality by transforming Rician noise into Gaussian noise, significantly enhancing the visualization needed for accurate diagnosis.\n\n**Key insights:**\n- Introduction of Rician Denoising Diffusion Probabilistic Models (RDDPM) enhances sodium MRI images. \n- RDDPM outperforms traditional denoising methods in feature preservation. \n- A significant advancement in tackling low signal-to-noise ratio challenges in medical imaging.\n\nWhat are your thoughts on this research direction?\n\n#MachineLearning #ArtificialIntelligence #Research #Innovation', additional_kwargs={}, response_metadata={}, name='ContentCreator')]}
------------------------

'Post generation complete!'

## System Summary

This multi-agent system successfully:

### Content Team:
- **Researcher**: Finds and analyzes ML papers using Arxiv
- **Content Creator**: Generates platform-specific social media posts

### Verification Team:
- **Technical Verifier**: Ensures accuracy and prevents misrepresentation
- **Platform Verifier**: Checks style, format, and platform compliance

### Key Features:
- ✅ Multi-platform support (Twitter, LinkedIn, Instagram)
- ✅ Technical accuracy verification
- ✅ Platform-specific style compliance
- ✅ Coordinated multi-team workflow
- ✅ Automated quality assurance

The system ensures that ML research is accurately and engagingly shared on social media while maintaining scientific integrity and platform best practices.
